In [20]:
import json
import requests

import boto3 
bedrock = boto3.client(
    service_name='s3', 
    region_name='us-east-1'
)

s3 = boto3.client('s3')
bucket = 'ch-agent-assist-prompt-library-bucket'
file = 'prompts_library.json' 

response = s3.get_object(Bucket=bucket,Key=file)
content = response['Body'].read().decode('utf-8')
content

'{\r\n  "entity_extraction": {\r\n    "enrollment_form_claude": "Human: {conversation}\\nThe above conversation is an automated transcript between a call centre agent and an insurance subscriber or patient.\\nI want to extract few key entities like {entities}.\\nAll or some information may be present in this transcript.\\nExtract the entities that you are able to find from this piece of call transcript.\\nIn the above text, if you find any section that talks about appointment or schedule time and contains text like \'this week\', \'next week\', \'this Friday\', \'next Monday\' etc; please convert that phrase to the actual date considering the difference from today\'s date.\\nThe output would be a structured json with only the extracted fields.\\nJust print the exact output without any extra sentences at the end or beginning.\\nNo need to print any extra text.\\nAlso, do not generate an answer if that is not found in the transcript.",\r\n    "enrollment_form_llama2": ""\r\n  },\r\n  "te

In [21]:
type(content)

str

In [22]:
c = json.loads(content)

In [23]:
print(c)

{'entity_extraction': {'enrollment_form_claude': "Human: {conversation}\nThe above conversation is an automated transcript between a call centre agent and an insurance subscriber or patient.\nI want to extract few key entities like {entities}.\nAll or some information may be present in this transcript.\nExtract the entities that you are able to find from this piece of call transcript.\nIn the above text, if you find any section that talks about appointment or schedule time and contains text like 'this week', 'next week', 'this Friday', 'next Monday' etc; please convert that phrase to the actual date considering the difference from today's date.\nThe output would be a structured json with only the extracted fields.\nJust print the exact output without any extra sentences at the end or beginning.\nNo need to print any extra text.\nAlso, do not generate an answer if that is not found in the transcript.", 'enrollment_form_llama2': ''}, 'text_summarization': {'summary_llama2': "Instruction:

In [24]:
c.keys()

dict_keys(['entity_extraction', 'text_summarization', 'insight_generation'])

In [32]:
c['insight_generation'].keys()

dict_keys(['insights_claude', 'insights_llama2'])

In [25]:
c['entity_extraction'].keys()

dict_keys(['enrollment_form_claude', 'enrollment_form_llama2'])

In [26]:
c['entity_extraction']['enrollment_form_claude']

"Human: {conversation}\nThe above conversation is an automated transcript between a call centre agent and an insurance subscriber or patient.\nI want to extract few key entities like {entities}.\nAll or some information may be present in this transcript.\nExtract the entities that you are able to find from this piece of call transcript.\nIn the above text, if you find any section that talks about appointment or schedule time and contains text like 'this week', 'next week', 'this Friday', 'next Monday' etc; please convert that phrase to the actual date considering the difference from today's date.\nThe output would be a structured json with only the extracted fields.\nJust print the exact output without any extra sentences at the end or beginning.\nNo need to print any extra text.\nAlso, do not generate an answer if that is not found in the transcript."

In [29]:
c['entity_extraction']['enrollment_form_claude'].format(conversation='conversation_1',entities='name,demographics')

"Human: conversation_1\nThe above conversation is an automated transcript between a call centre agent and an insurance subscriber or patient.\nI want to extract few key entities like name,demographics.\nAll or some information may be present in this transcript.\nExtract the entities that you are able to find from this piece of call transcript.\nIn the above text, if you find any section that talks about appointment or schedule time and contains text like 'this week', 'next week', 'this Friday', 'next Monday' etc; please convert that phrase to the actual date considering the difference from today's date.\nThe output would be a structured json with only the extracted fields.\nJust print the exact output without any extra sentences at the end or beginning.\nNo need to print any extra text.\nAlso, do not generate an answer if that is not found in the transcript."

In [31]:
def get_prompt(bucket,file,prompt_category,required_prompt,conversation,entities):
    s3 = boto3.client('s3') 
    response = s3.get_object(Bucket=bucket,Key=file)
    content = response['Body'].read().decode('utf-8')
    json_content = json.loads(content)
    prompt = json_content[prompt_category][required_prompt].format(conversation=conversation,entities=entities)
    
    return prompt
    
prompt = get_prompt(bucket,file,'entity_extraction','enrollment_form_claude','conversation_1','name,age etc.')
print(prompt)  # Output should be the greeting prompt from the 'customer_service' category

Human: conversation_1
The above conversation is an automated transcript between a call centre agent and an insurance subscriber or patient.
I want to extract few key entities like name,age etc..
All or some information may be present in this transcript.
Extract the entities that you are able to find from this piece of call transcript.
In the above text, if you find any section that talks about appointment or schedule time and contains text like 'this week', 'next week', 'this Friday', 'next Monday' etc; please convert that phrase to the actual date considering the difference from today's date.
The output would be a structured json with only the extracted fields.
Just print the exact output without any extra sentences at the end or beginning.
No need to print any extra text.
Also, do not generate an answer if that is not found in the transcript.


In [16]:
print(prompt.format('conversation_1','Tuesday'))

Human: "{conv3}"

The above conversation is an automated transcript between a call centre agent and an insurance subscriber or patient. I want to extract a few key entities like the name of the patient, status of insurance, insurance number, demographic details, etc. All or some information may be present in this transcript. Extract the entities that you are able to find from this piece of call transcript. In the above text, if you find any section that talks about an appointment or schedule time and contains text like 'this week', 'next week', 'this Friday', 'next Monday', etc; please convert that phrase to the actual date considering the difference from today's date which is {today}.

The output would be a structured json with only the extracted fields. Just print the exact output. No need to print any extra text. Also, do not generate an answer if that is not found in the transcript.

Assistant:
